## reference link (PYEEG)
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3070217/

In [1]:
import scipy.io as sio
import glob
import numpy
import time

In [2]:
def load_patient_train_data(paths):
# Load training data for patient


    X = []
    Y = []

    print('...loading train data')
    start = time.time()

    for path in sorted(glob.glob(paths), key=numericalSort):
        X.append(sio.loadmat(path))
        Y.append(int(path[-5]))
    
    Y = numpy.array(Y)
    print('time elapsed: %s sec' %(time.time() - start))
    
    
    return(X, Y)

In [3]:
def load_patient_test_data(paths):
# Load training data for patient


    X = []
    file_array = []

    print('...loading test data')
    start = time.time()

    for path in sorted(glob.glob(paths), key=numericalSort):
        X.append(sio.loadmat(path))
        file_array.append(os.path.split(path)[1])
    
    print('time elapsed: %s sec' %(time.time() - start))
    
    
    return(X, file_array)

In [4]:
# The numericalSort function splits out any digits in a filename, 
# turns it into an actual number, and returns the result for sorting

import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [11]:
def new2_n_sample_fft(X, samples, index):
    
    start = time.time()
    num_freq= 240000/2 + 1
    channels = 16
    X_fft = numpy.zeros([samples, num_freq, channels])

    for sample in xrange(samples):
            X_fft[sample,:,:] = numpy.fft.rfft(X[sample+index]['dataStruct']['data'][0][0], axis=0)
            
    X_fft = numpy.absolute(X_fft)
    
    print('time elapsed: %s sec' %(time.time() - start))
    return X_fft

In [6]:
# Find samples of training set X that contain no data, or consist entirely of zeros
def find_zero_index(X):
    zero_index = []
    print('...locating zero-data')
    
    for i in xrange(len(X)):
        if numpy.sum(numpy.absolute(X[i]['dataStruct']['data'][0][0])) == 0:
            zero_index.append(i)
    
    return zero_index

In [7]:
# remove all-zero data
# X must be list, Y can be numpy array which is cast to a list, and cast back to numpy array upon return
def remove_zero_data(X, Y):
    
    zero_index = find_zero_index(X)
    index_correction = 0
    # index_correction is needed because every time element is deleted from a list the following elements are shifted
    # EX: if 2nd element is deleted, the 3rd element becomes the 2nd, the 4th becomes the 3rd, etc.
    Y = list(Y)
    print('...removing zero-data')
    
    for i in xrange(len(zero_index)):

        del X[zero_index[i] - index_correction]
        del Y[zero_index[i] - index_correction]

        index_correction += 1
   
    Y = numpy.array(Y)
    return(X, Y)

In [8]:
# load training data
X_train1, Y_train1 = load_patient_train_data('F:/Kaggle/Seizure Prediction/train_1/*.mat')

...loading train data
time elapsed: 115.257999897 sec


In [9]:
X_train1, Y_train1 = remove_zero_data(X_train1, Y_train1)

...locating zero-data
...removing zero-data


In [12]:
# compute PSI and RIR

# frequency bands of interest
# delta, theta, alpha, beta, gamma respectively 
frequency_bins = numpy.array([0.5, 4, 7, 12, 30, 100])

bins = 10
X_train1_len = len(X_train1)
channels = 16

N = 240000/2 + 1
# sample spacing
T = 1.0 / 400.0
xf = numpy.linspace(0.0, 1.0/(2.0*T), N/2)

# create list (1 to 1300)
X_train1_index = numpy.linspace(0, X_train1_len-1, X_train1_len-1)
# Separate created list into 10 bins. The size of these bins is used to perform batch computation of FFT
# and half-power frequency calulation
# Calculation is far from optimized
data_bins = numpy.histogram(X_train1_index, bins=bins)
index = 0
PSI = numpy.zeros([X_train1_len, len(frequency_bins)-1, channels])
RIR = numpy.zeros([X_train1_len, len(frequency_bins)-1, channels])


for bin in xrange(bins):
    print('batch number: %s' %(bin))
    fft_batch = new2_n_sample_fft(X_train1, data_bins[0][bin], index)
    
    for i in xrange(len(frequency_bins)-1):
        bin_start = numpy.where((frequency_bins[i+1] >= xf) & (xf >= frequency_bins[i]))[0][0]
        bin_end = numpy.where((frequency_bins[i+1] >= xf) & (xf >= frequency_bins[i]))[0][-1]
        PSI[index:index + data_bins[0][bin], i, :] = numpy.real(numpy.sum(numpy.absolute(fft_batch[:, bin_start: bin_end, :]), 
                                                                       axis=1))
    
    
    PSI_batch_sum = numpy.sum(PSI[index:index + data_bins[0][bin], :, :], axis=1)
    PSI_batch_sum_array = numpy.ones([data_bins[0][bin], len(frequency_bins) -1, channels])
    
    for i in xrange(len(frequency_bins) -1):
        PSI_batch_sum_array[:, i, :] = PSI_batch_sum 
    
    RIR[index:index + data_bins[0][bin], :, :] = PSI[index:index + data_bins[0][bin], :, :]/PSI_batch_sum_array
    index = index + data_bins[0][bin]

batch number: 0


C:\Users\Ikes\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: ComplexWarning: Casting complex values to real discards the imaginary part


time elapsed: 13.1819999218 sec
batch number: 1
time elapsed: 13.1590001583 sec
batch number: 2
time elapsed: 13.4600000381 sec
batch number: 3
time elapsed: 13.2999999523 sec
batch number: 4
time elapsed: 12.6069998741 sec
batch number: 5
time elapsed: 11.6429998875 sec
batch number: 6
time elapsed: 12.3410000801 sec
batch number: 7
time elapsed: 13.117000103 sec
batch number: 8
time elapsed: 13.378000021 sec
batch number: 9
time elapsed: 13.5769999027 sec


In [13]:
numpy.save('F:/Kaggle/Seizure Prediction/features/FEATURE_train1_power_spectral_intensity', PSI)
numpy.save('F:/Kaggle/Seizure Prediction/features/FEATURE_train1_relative_intensity_ratio', RIR)